In [1]:
from lxml.etree import HTML
from lxml import etree
import requests
import json
import pandas

df = pandas.read_html(
    requests.get("https://en.wiktionary.org/wiki/Wiktionary:Frequency_lists/Finnish/Press_data").text
)[0]

words = df[df["PoS"] == "noun"]["word"]

In [8]:
import re

doc = {}
for word in words:
    if word == "tilanne":
        continue
    if len(doc) >= 100:
        break
    for table in HTML(requests.get(f"https://en.wiktionary.org/wiki/{word}").text).xpath("//table"):
        if "fi-decl" in table.attrib.get("class", ""):
            doc[word] = {"s": {"nom": word}}
            for span in table.xpath("//span"):
                m = re.match(".*? (\w+)(?:\/\/\w+)?\|(\w)-form-of", span.attrib.get("class", ""))
                if m and "lang-fi" in span.attrib.get("class", ""):
                    case, quant = m.groups()
                    if quant not in doc[word]:
                        doc[word][quant] = {}
                    doc[word][quant][case] = "".join(span.itertext())
            doc[word]["s"]["_stem"] = re.sub('ss[aä]$', '', doc[word]["s"]["ine"])
            doc[word]["p"]["_stem"] = re.sub('ss[aä]$', '', doc[word]["p"]["ine"])

In [9]:
for word, d in doc.items():
    print(word, d["s"]["_stem"])
    print(word, d["p"]["_stem"])

vuosi vuode
vuosi vuosi
markka marka
markka markoi
maa maa
maa mai
asia asia
asia asioi
prosentti prosenti
prosentti prosentei
ihminen ihmise
ihminen ihmisi
mies miehe
mies miehi
aika aja
aika ajoi
työ työ
työ töi
mieli miele
mieli mieli
kaupunki kaupungi
kaupunki kaupungei
osa osa
osa osi
alue aluee
alue aluei
lapsi lapse
lapsi lapsi
päivä päivä
päivä päivi
hallitus hallitukse
hallitus hallituksi
kunta kunna
kunta kunni
nainen naise
nainen naisi
viikko viiko
viikko viikoi
luku luvu
luku luvui
kerta kerra
kerta kerroi
yritys yritykse
yritys yrityksi
alku alu
alku alui
paikka paika
paikka paikoi
presidentti presidenti
presidentti presidentei
tieto tiedo
tieto tiedoi
loppu lopu
loppu lopui
tapa tava
tapa tavoi
maailma maailma
maailma maailmoi
määrä määrä
määrä määri
puheenjohtaja puheenjohtaja
puheenjohtaja puheenjohtaji
poliisi poliisi
poliisi poliisei
kisa kisa
kisa kisoi
auto auto
auto autoi
toiminta toiminna
toiminta toiminnoi
tulos tulokse
tulos tuloksi
raha raha
raha rahoi
päätös p

In [10]:
with open("public/nouns.json", "w") as f:
    f.write(json.dumps(doc, indent=4))

In [ ]:
import re
re.sub("ss[aä]$", "", "miehassä")